In [5]:
%load_ext autoreload
%autoreload
%cd /home/jupyter/MathVGerrmandering_CMAT_2021/
notebook = True
skip_inputs = 'y'
%run -i redistricter

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/jupyter/MathVGerrmandering_CMAT_2021
Get crosswalks_TX_2010_tabblock   ... tabblock table exists ... success
Get assignments_TX_2020_tabblock  ... tabblock table exists ... success
Get shapes_TX_2020_tabblock       ... tabblock table exists ... success
Get census_TX_2020_tabblock       ... tabblock table exists ... success
Get elections_TX_2020_tabblock    ... tabblock table exists ... success
Get nodes_TX_2020_cntyvtd_cd      ... cntyvtd table exists ... success
Get graph_TX_2020_cntyvtd         ... gpickle exists ... success
hashseed != 0 so results are NOT reproducible and will NOT be saved to BigQuery
I will run seeds [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]
cmat-315920.results.TX_2020_cntyvtd_cd_2021_09_06_03_35_51_results


NotFound: 404 Not found: Table cmat-315920:results.TX_2020_cntyvtd_cd_2021_09_06_03_35_51_seed_0039_summary was not found in location US

(job ID: f0672f8a-b101-4e1e-a82c-8363be156acf)

                                             -----Query Job SQL Follows-----                                             

    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |
   1:
   2:select
   3:    B.seed,
   4:    B.plan,
   5:    C.cd,
   6:    max(B.hash) as hash_plan,
   7:    max(B.pop_imbalance) as pop_imbalance_plan,
   8:    max(B.polsby_popper) as polsby_popper_plan,
   9:    max(C.polsby_popper) as polsby_popper_district,
  10:    max(C.aland) as aland,
  11:    max(C.total_pop) as total_pop,
  12:    max(C.total_pop) / sum(E.aland) as density,
  13:    sum(E.total_1race) as total_1race,
  14:    sum(E.total_white) as total_white,
  15:    sum(E.total_black) as total_black,
  16:    sum(E.total_native) as total_native,
  17:  

In [3]:
str(root_path)+'/'

'/home/jupyter/'

In [1]:
1+1

2

In [77]:
results_loc

PosixPath('/home/jupyter/results/TX_2020_cntyvtd_cd/2021_09_05_20_44_29')

In [17]:
f = pathlib.Path('/home/jupyter/redistricting_data/crosswalks/TX')
# f.mkdir()
str(f)

gcsclient.

'/home/jupyter/redistricting_data/crosswalks/TX'

In [129]:
str(G.crosswalks.pq).replace(str(root_loc), 'gs://bucket')

'gs://bucket/redistricting_data/crosswalks/TX/crosswalks_TX_2010.parquet'

/home/jupyter/MathVGerrmandering_CMAT_2021


In [13]:
G.crosswalks.zip


# pq.with_suffix('.temp')

NameError: name 'G' is not defined

In [83]:
dir(data_gcs)
data_gcs.path

'/b/redistricting_data'

In [101]:
df = read_table(G.crosswalks.tbl)

In [106]:
df.to_parquet(root_loc / 'temp.parquet')

In [108]:
df.size

3109179

In [103]:
df2 = pd.read_parquet(root_loc / 'temp.pq')

In [109]:
df2.size

3109179

## 

In [107]:
f = data_gcs + G.crosswalks.pq.name
df2.to_parquet(f)

In [105]:
df.shape, df2.shape

((1036393, 3), (1036393, 3))

In [104]:
df2

,geoid_2010,geoid_2020,aland_prop
0,480019501001001,480019501001001,0.885936
1,480019501001001,480019501001048,0.114064
2,480019501002009,480019501002001,0.010068
3,480019501002017,480019501002001,0.999556
4,480019501002225,480019501002001,0.000275
...,...,...,...
1036388,485079503025019,485079503025020,1.000000
1036389,485079503025020,485079503025021,1.000000
1036390,485079503025021,485079503025022,1.000000
1036391,485079503025025,485079503025023,1.000000


In [98]:
data_gcs = f'gs://redistricting_data/'
f = data_gcs + G.crosswalks.pq.name
f
bqclient.extract_table(G.crosswalks.tbl, data_gcs + G.crosswalks.pq.name).result()

'gs://redistricting_data/crosswalks_TX_2010.parquet'

In [97]:
bqclient.extract_table?

Signature:
bqclient.extract_table(
    source,
    destination_uris,
    job_id=None,
    job_id_prefix=None,
    location=None,
    project=None,
    job_config=None,
    retry=<google.api_core.retry.Retry object at 0x7f3d14ec8310>,
    timeout=None,
    source_type='Table',
)
Docstring:
Start a job to extract a table into Cloud Storage files.

See
https://cloud.google.com/bigquery/docs/reference/rest/v2/Job#jobconfigurationextract

Args:
    source (Union[                 google.cloud.bigquery.table.Table,                 google.cloud.bigquery.table.TableReference,                 google.cloud.bigquery.model.Model,                 google.cloud.bigquery.model.ModelReference,                 src,             ]):
        Table or Model to be extracted.
    destination_uris (Union[str, Sequence[str]]):
        URIs of Cloud Storage file(s) into which table data is to be
        extracted; in format
        ``gs://<bucket_name>/<object_name_or_glob>``.

Keyword Arguments:
    job_id (Opti

In [99]:
f = root_loc / 'crosswalks_TX_2010.parquet'
f
df = pd.read_parquet(f)

In [100]:
df

,geoid_2010,geoid_2020,aland_prop
38,480019501001000,480019501001000,1.000000
42,480019501001003,480019501001001,1.000000
39,480019501001001,480019501001001,0.885936
45,480019501001006,480019501001002,1.000000
43,480019501001004,480019501001003,1.000000
...,...,...,...
1036385,485079503025019,485079503025020,1.000000
1036386,485079503025020,485079503025021,1.000000
1036387,485079503025021,485079503025022,1.000000
1036391,485079503025025,485079503025023,1.000000


In [65]:
'gs://'+b.name

'gs://math_for_unbiased_maps_tx'

In [61]:
b = gcsclient.get_bucket(f'math_for_unbiased_maps_tx')
df = pd.DataFrame()
# df.to_parquet(b.blob('test'))

t = b.blob('test/test2')
# df.to_parquet(t)
# t.path
# g://t.name
dir(t)
# t._get_download_url(gcsclient)

# t.__name__
# t.path_helper?

'https://storage.googleapis.com/download/storage/v1/b/math_for_unbiased_maps_tx/o/test%2Ftest2?alt=media'

In [36]:
b = gcsclient.get_bucket(f'math_for_unbiased_maps_tx')
c = b.blob('test')
c.upload_from_filenameid(root_loc / 'git.ipynb')

AttributeError: 'Blob' object has no attribute 'upload_from_filenameid'

In [53]:
proj_id = 'cmat-315920'
root_gcs = gcsclient.get_bucket(f'{proj_id}-bucket')
# root_gcs.get_blob

b = root_gcs.blob('redistricting_data/hi2.txt')#.upload_from_string('hello world')


b.blob('sub')
# gcsclient.create_bucket?


# get_bucket(f'{proj_id}-bucket')
# data_gcs = root_gcs.get_blob('redistricting_data/hi.txt')
# data_gcs.upload_from_string('hello world')

AttributeError: 'Blob' object has no attribute 'blob'

In [36]:
import google.cloud, pandas as pd
cred, proj = google.auth.default(scopes=["https://www.googleapis.com/auth/cloud-platform"])

proj_id = 'cmat-315920'
user_name = 'cook'
timestamp = str(pd.Timestamp.now().round("s")).replace(' ','_').replace('-','_').replace(':','_')

bqclient   = google.cloud.bigquery.Client(credentials=cred, project=proj)
bqdata     = bqclient.get_dataset(f'redistricting_data')
bqresults  = bqclient.get_dataset(f'redistricting_results_{user_name}')

L = [x for x in bqclient.list_tables(bqdata)]
dir(L[0])
t = L[0]
t.table_id
bqdata.


# gcsclient  = google.cloud.storage.Client(credentials=cred, project=proj)
# gcsresults = gcsclient.get_bucket(f'redistricting_results')

# gcsclient.create_bucket('redistricting_results')

'assignments_TX_2020_tabblock'

In [52]:
# bqproj = 
list(bqclient.list_datasets())
bqclient.project

'cmat-315920'

In [ ]:
bqclient   = bigquery.Client(credentials=cred, project=proj)

In [47]:
gcsclient = google.cloud.storage.Client(project=proj_id)
list(gcsclient.list_buckets())
gcsbucket = gcsclient.get_bucket(f'{proj_id}-bucket')
folder = gcsbucket.blob('test')
gcsbucket.exists()
folder.exists()
gcsbucket.blob?

Signature:
gcsbucket.blob(
    blob_name,
    chunk_size=None,
    encryption_key=None,
    kms_key_name=None,
    generation=None,
)
Docstring:
Factory constructor for blob object.

.. note::
  This will not make an HTTP request; it simply instantiates
  a blob object owned by this bucket.

:type blob_name: str
:param blob_name: The name of the blob to be instantiated.

:type chunk_size: int
:param chunk_size: The size of a chunk of data whenever iterating
                   (in bytes). This must be a multiple of 256 KB per
                   the API specification.

:type encryption_key: bytes
:param encryption_key:
    (Optional) 32 byte encryption key for customer-supplied encryption.

:type kms_key_name: str
:param kms_key_name:
    (Optional) Resource name of KMS key used to encrypt blob's content.

:type generation: long
:param generation: (Optional) If present, selects a specific revision of
                   this object.

:rtype: :class:`google.cloud.storage.blob.Blob`
:return

In [41]:
gcsbucket = gcsclient.get_bucket(f'{proj_id}-bucket')
list(gcsbucket.list_blobs())

[<Blob: cmat-315920-bucket, TX_2020_cntyvtd_cd_seed_0200/.ipynb_checkpoints/TX_2020_cntyvtd_cd_seed_0200_map-checkpoint.html, 1630589527875872>,
 <Blob: cmat-315920-bucket, TX_2020_cntyvtd_cd_seed_0200/.ipynb_checkpoints/TX_2020_cntyvtd_cd_seed_0200_results-checkpoint.csv, 1630589527990923>,
 <Blob: cmat-315920-bucket, TX_2020_cntyvtd_cd_seed_0200/.ipynb_checkpoints/TX_2020_cntyvtd_cd_seed_0200_summary-checkpoint.csv, 1630589528068479>,
 <Blob: cmat-315920-bucket, TX_2020_cntyvtd_cd_seed_0200/TX_2020_cntyvtd_cd_seed_0200_graph.gpickle, 1630589527627943>,
 <Blob: cmat-315920-bucket, TX_2020_cntyvtd_cd_seed_0200/TX_2020_cntyvtd_cd_seed_0200_map.html, 1630589527377459>,
 <Blob: cmat-315920-bucket, TX_2020_cntyvtd_cd_seed_0200/TX_2020_cntyvtd_cd_seed_0200_results.csv, 1630589526999672>,
 <Blob: cmat-315920-bucket, TX_2020_cntyvtd_cd_seed_0200/TX_2020_cntyvtd_cd_seed_0200_summary.csv, 1630589527466920>,
 <Blob: cmat-315920-bucket, TX_2020_cntyvtd_cd_seed_0201/TX_2020_cntyvtd_cd_seed_0201_gr

In [ ]:
root_gcs = gcsclient.get_bucket(f'{proj_id}-bucket')
data_gcs = root_gcs.get_blob('redistricting_data')


In [30]:
class Results(Base):
    
    



timestamp = str(pd.Timestamp.now().round("s")).replace(' ','_').replace('-','_').replace(':','_')

bq_results
gcs_results


results_stem = f'{G.gpickle.stem[6:]}/{timestamp}'

results_loc = root_path / f'results/{results_stem}'
results_gcs = f'gc://{proj_id}-bucket/results/{results_stem}'
results_bq  = f'{proj_id}.{results_stem}'

results_loc.mkdir(parents=True, exist_ok=True)

gcsclient = storage.Client(project=proj_id)

gcsbucket = gcsclient.get_bucket('buckets')
blob = bucket.blob('some/folder/name/')
bqclient.create_dataset(results_bq, exists_ok=True)
results_gcs

TypeError: mkdir() got an unexpected keyword argument 'exists_ok'

In [29]:
bqclient.create_dataset?

Signature:
bqclient.create_dataset(
    dataset,
    exists_ok=False,
    retry=<google.api_core.retry.Retry object at 0x7f066a8d1210>,
    timeout=None,
)
Docstring:
API call: create the dataset via a POST request.

See
https://cloud.google.com/bigquery/docs/reference/rest/v2/datasets/insert

Args:
    dataset (Union[                 google.cloud.bigquery.dataset.Dataset,                 google.cloud.bigquery.dataset.DatasetReference,                 str,             ]):
        A :class:`~google.cloud.bigquery.dataset.Dataset` to create.
        If ``dataset`` is a reference, an empty dataset is created
        with the specified ID and client's default location.
    exists_ok (Optional[bool]):
        Defaults to ``False``. If ``True``, ignore "already exists"
        errors when creating the dataset.
    retry (Optional[google.api_core.retry.Retry]):
        How to retry the RPC.
    timeout (Optional[float]):
        The number of seconds to wait for the underlying HTTP transport


In [10]:
root_path

PosixPath('/home/jupyter')

In [1]:
from src import *

getting states


In [2]:
bqclient.create_dataset?

Signature:
bqclient.create_dataset(
    dataset,
    exists_ok=False,
    retry=<google.api_core.retry.Retry object at 0x7f066a8d1210>,
    timeout=None,
)
Docstring:
API call: create the dataset via a POST request.

See
https://cloud.google.com/bigquery/docs/reference/rest/v2/datasets/insert

Args:
    dataset (Union[                 google.cloud.bigquery.dataset.Dataset,                 google.cloud.bigquery.dataset.DatasetReference,                 str,             ]):
        A :class:`~google.cloud.bigquery.dataset.Dataset` to create.
        If ``dataset`` is a reference, an empty dataset is created
        with the specified ID and client's default location.
    exists_ok (Optional[bool]):
        Defaults to ``False``. If ``True``, ignore "already exists"
        errors when creating the dataset.
    retry (Optional[google.api_core.retry.Retry]):
        How to retry the RPC.
    timeout (Optional[float]):
        The number of seconds to wait for the underlying HTTP transport


In [40]:
%autoreload
from src import *

nodes_tbl = f'{proj_id}.redistricting_data.nodes_TX_2020_cntyvtd_cd'
mcmc_tbl  = f'{proj_id}.redistricting_results_cook.TX_2020_cntyvtd_cd_seed_0000'
seeds = np.arange(16)
A = Analysis(nodes=nodes_tbl, mcmc=mcmc_tbl, seeds=seeds)
A.tbl = f'{proj_id}.redistricting_results_cook.TX_2020_cntyvtd_cd_2021_09_05_01_50_26'
A.fetch_results()
A.save_results()
# A.file

,seed,plan,cd,hash_plan,pop_imbalance_plan,polsby_popper_plan,polsby_popper_district,aland,total_pop,density,total_1race,total_white,total_black,total_native,total_asian,total_pacific,total_other,total_2ormorerace,total_2race,total_white_black,total_white_native,total_white_asian,total_white_pacific,total_white_other,total_black_native,total_black_asian,total_black_pacific,total_black_other,total_native_asian,total_native_pacific,total_native_other,total_asian_pacific,total_asian_other,total_pacific_other,total_3race,total_white_black_native,total_white_black_asian,total_white_black_pacific,total_white_black_other,total_white_native_asian,total_white_native_pacific,total_white_native_other,total_white_asian_pacific,total_white_asian_other,total_white_pacific_other,total_black_native_asian,total_black_native_pacific,total_black_native_other,total_black_asian_pacific,total_black_asian_other,total_black_pacific_other,total_native_asian_pacific,total_native_asian_other,total_native_pacific_other,total_asian_pacific_other,total_4race,total_white_black_native_asian,total_white_black_native_pacific,total_white_black_native_other,total_white_black_asian_pacific,total_white_black_asian_other,total_white_black_pacific_other,total_white_native_asian_pacific,total_white_native_asian_other,total_white_native_pacific_other,total_white_asian_pacific_other,total_black_native_asian_pacific,total_black_native_asian_other,total_black_native_pacific_other,total_black_asian_pacific_other,total_native_asian_pacific_other,total_5race,total_white_black_native_asian_pacific,total_white_black_native_asian_other,total_white_black_native_pacific_other,total_white_black_asian_pacific_other,total_white_native_asian_pacific_other,total_black_native_asian_pacific_other,total_6race,total_white_black_native_asian_pacific_other,hisp_pop,nonhisp_pop,nonhisp_1race,nonhisp_white,nonhisp_black,nonhisp_native,nonhisp_asian,nonhisp_pacific,nonhisp_other,nonhisp_2ormorerace,nonhisp_2race,nonhisp_white_black,nonhisp_white_native,nonhisp_white_asian,nonhisp_white_pacific,nonhisp_white_other,nonhisp_black_native,nonhisp_black_asian,nonhisp_black_pacific,nonhisp_black_other,nonhisp_native_asian,nonhisp_native_pacific,nonhisp_native_other,nonhisp_asian_pacific,nonhisp_asian_other,nonhisp_pacific_other,nonhisp_3race,nonhisp_white_black_native,nonhisp_white_black_asian,nonhisp_white_black_pacific,nonhisp_white_black_other,nonhisp_white_native_asian,nonhisp_white_native_pacific,nonhisp_white_native_other,nonhisp_white_asian_pacific,nonhisp_white_asian_other,nonhisp_white_pacific_other,nonhisp_black_native_asian,nonhisp_black_native_pacific,nonhisp_black_native_other,nonhisp_black_asian_pacific,nonhisp_black_asian_other,nonhisp_black_pacific_other,nonhisp_native_asian_pacific,nonhisp_native_asian_other,nonhisp_native_pacific_other,nonhisp_asian_pacific_other,nonhisp_4race,nonhisp_white_black_native_asian,nonhisp_white_black_native_pacific,nonhisp_white_black_native_other,nonhisp_white_black_asian_pacific,nonhisp_white_black_asian_other,nonhisp_white_black_pacific_other,nonhisp_white_native_asian_pacific,nonhisp_white_native_asian_other,nonhisp_white_native_pacific_other,nonhisp_white_asian_pacific_other,nonhisp_black_native_asian_pacific,nonhisp_black_native_asian_other,nonhisp_black_native_pacific_other,nonhisp_black_asian_pacific_other,nonhisp_native_asian_pacific_other,nonhisp_5race,nonhisp_white_black_native_asian_pacific,nonhisp_white_black_native_asian_other,nonhisp_white_black_native_pacific_other,nonhisp_white_black_asian_pacific_other,nonhisp_white_native_asian_pacific_other,nonhisp_black_native_asian_pacific_other,nonhisp_6race,nonhisp_white_black_native_asian_pacific_other,o17_pop,o17_1race,o17_white,o17_black,o17_native,o17_asian,o17_pacific,o17_other,o17_2ormorerace,o17_2race,o17_white_black,o17_white_native,o17_white_asian,o17_white_pacific,o17_white_other,o17_black_native,o17_black_asian,o17_black_pacific,o17_black_other,o17_native_asian,o17_native_pacific,o17_n

In [53]:
dir(A)
'_'.join(G.gpickle.stem.split('_')[1:])

'TX_2020_cntyvtd_cd'

In [36]:
A.tbl.split('.')[-1]

'TX_2020_cntyvtd_cd_2021_09_05_01_50_26'

PosixPath('/home/jupyter/results/TX_2020_cntyvtd_cd_2021_09_05_01_50_26.parquet')

In [39]:
A.results.to_parquet(f"gs://{proj_id}-bucket/{A.tbl.split('.')[-1]}")

In [32]:
A.results.shape

(484462, 852)

In [29]:
self=A
A.tbl
root_path / f'results/{self.tbl.split(".")[-1]}.parquet'

PosixPath('/home/jupyter/results/TX_2020_cntyvtd_cd_2021_09_05_01_50_26.parquet')

In [143]:
%autoreload
from src import *

idx = ['seed', 'plan', 'cd']
for col in idx:
    df[col] = rjust(df[col])
df.sort_values(idx, inplace=True)
file = pathlib.Path('/home/jupyter/results/TX_2020_cntyvtd_cd_2021_09_05_01_50_26')
file.mkdir(parents=True, exist_ok=True)
df.to_parquet(file, index=False, partition_cols='seed')

In [139]:
df.head(10)

,seed,plan,cd,hash_plan,pop_imbalance_plan,polsby_popper_plan,polsby_popper_district,aland,total_pop,density,total_1race,total_white,total_black,total_native,total_asian,total_pacific,total_other,total_2ormorerace,total_2race,total_white_black,total_white_native,total_white_asian,total_white_pacific,total_white_other,total_black_native,total_black_asian,total_black_pacific,total_black_other,total_native_asian,total_native_pacific,total_native_other,total_asian_pacific,total_asian_other,total_pacific_other,total_3race,total_white_black_native,total_white_black_asian,total_white_black_pacific,total_white_black_other,total_white_native_asian,total_white_native_pacific,total_white_native_other,total_white_asian_pacific,total_white_asian_other,total_white_pacific_other,total_black_native_asian,total_black_native_pacific,total_black_native_other,total_black_asian_pacific,total_black_asian_other,total_black_pacific_other,total_native_asian_pacific,total_native_asian_other,total_native_pacific_other,total_asian_pacific_other,total_4race,total_white_black_native_asian,total_white_black_native_pacific,total_white_black_native_other,total_white_black_asian_pacific,total_white_black_asian_other,total_white_black_pacific_other,total_white_native_asian_pacific,total_white_native_asian_other,total_white_native_pacific_other,total_white_asian_pacific_other,total_black_native_asian_pacific,total_black_native_asian_other,total_black_native_pacific_other,total_black_asian_pacific_other,total_native_asian_pacific_other,total_5race,total_white_black_native_asian_pacific,total_white_black_native_asian_other,total_white_black_native_pacific_other,total_white_black_asian_pacific_other,total_white_native_asian_pacific_other,total_black_native_asian_pacific_other,total_6race,total_white_black_native_asian_pacific_other,hisp_pop,nonhisp_pop,nonhisp_1race,nonhisp_white,nonhisp_black,nonhisp_native,nonhisp_asian,nonhisp_pacific,nonhisp_other,nonhisp_2ormorerace,nonhisp_2race,nonhisp_white_black,nonhisp_white_native,nonhisp_white_asian,nonhisp_white_pacific,nonhisp_white_other,nonhisp_black_native,nonhisp_black_asian,nonhisp_black_pacific,nonhisp_black_other,nonhisp_native_asian,nonhisp_native_pacific,nonhisp_native_other,nonhisp_asian_pacific,nonhisp_asian_other,nonhisp_pacific_other,nonhisp_3race,nonhisp_white_black_native,nonhisp_white_black_asian,nonhisp_white_black_pacific,nonhisp_white_black_other,nonhisp_white_native_asian,nonhisp_white_native_pacific,nonhisp_white_native_other,nonhisp_white_asian_pacific,nonhisp_white_asian_other,nonhisp_white_pacific_other,nonhisp_black_native_asian,nonhisp_black_native_pacific,nonhisp_black_native_other,nonhisp_black_asian_pacific,nonhisp_black_asian_other,nonhisp_black_pacific_other,nonhisp_native_asian_pacific,nonhisp_native_asian_other,nonhisp_native_pacific_other,nonhisp_asian_pacific_other,nonhisp_4race,nonhisp_white_black_native_asian,nonhisp_white_black_native_pacific,nonhisp_white_black_native_other,nonhisp_white_black_asian_pacific,nonhisp_white_black_asian_other,nonhisp_white_black_pacific_other,nonhisp_white_native_asian_pacific,nonhisp_white_native_asian_other,nonhisp_white_native_pacific_other,nonhisp_white_asian_pacific_other,nonhisp_black_native_asian_pacific,nonhisp_black_native_asian_other,nonhisp_black_native_pacific_other,nonhisp_black_asian_pacific_other,nonhisp_native_asian_pacific_other,nonhisp_5race,nonhisp_white_black_native_asian_pacific,nonhisp_white_black_native_asian_other,nonhisp_white_black_native_pacific_other,nonhisp_white_black_asian_pacific_other,nonhisp_white_native_asian_pacific_other,nonhisp_black_native_asian_pacific_other,nonhisp_6race,nonhisp_white_black_native_asian_pacific_other,o17_pop,o17_1race,o17_white,o17_black,o17_native,o17_asian,o17_pacific,o17_other,o17_2ormorerace,o17_2race,o17_white_black,o17_white_native,o17_white_asian,o17_white_pacific,o17_white_other,o17_black_native,o17_black_asian,o17_black_pacific,o17_black_other,o17_native_asian,o17_native_pacific,o17_n

In [92]:
district_type = 'cd'

tbl = f'cmat-315920.redistricting_results_cook.TX_2020_cntyvtd_cd_seed'
s = "\nunion all\n"
seeds = [s for s in range (100, 116) if s not in [104, 110]]
summary_stack = s.join([f'select * from {tbl}_{str(seed).rjust(4, "0")}_summary' for seed in seeds])
stats_stack   = s.join([f'select * from {tbl}_{str(seed).rjust(4, "0")}_stats'   for seed in seeds])
plans_stack   = s.join([f'select * from {tbl}_{str(seed).rjust(4, "0")}_plans'   for seed in seeds])

query = f"""
select
    B.seed,
    B.plan,
    C.{district_type},
    B.pop_imbalance,
    B.polsby_popper as polsby_popper_plan,
    C.polsby_popper as polsby_popper_district,
    C.total_pop
from (
    select
        *
    from (
        select
            *,
            row_number() over (partition by A.hash order by plan asc, seed asc) as r
        from (
            {summary_stack}
            ) as A
        )
    where r = 1
    ) as B
join (
    select
        *
    from (
        {stats_stack}
        )
    ) as C
on
    B.seed = C.seed and B.plan = C.plan
"""
df = run_query(query)
# 
df#['hash'].value_counts().sort_values(ascending=False)

,seed,plan,cd,pop_imbalance,polsby_popper_plan,polsby_popper_district,total_pop
0,105,1,01,88.179594,20.215682,19.400764,722363
1,105,2,01,84.423077,20.303012,19.400764,722363
2,105,3,01,80.639179,20.131629,19.400764,722363
3,105,4,01,80.639179,20.116386,19.400764,722363
4,105,5,01,80.639179,20.145664,19.400764,722363
...,...,...,...,...,...,...,...
16563,106,36,38,51.809094,18.371053,21.889269,843877
16564,106,37,38,51.369191,18.172050,21.889269,843877
16565,106,38,38,50.333710,18.533837,21.889269,843877
16566,106,39,38,50.333710,18.580757,21.889269,843877


In [101]:
district_type = 'cd'

nodes = f'cmat-315920.redistricting_data.nodes_TX_2020_cntyvtd_cd'
tbl = f'cmat-315920.redistricting_results_cook.TX_2020_cntyvtd_cd_seed'
s = "\nunion all\n"
seeds = [s for s in range (100, 116) if s not in [104, 110]]
summary_stack = s.join([f'select * from {tbl}_{str(seed).rjust(4, "0")}_summary' for seed in seeds])
stats_stack   = s.join([f'select * from {tbl}_{str(seed).rjust(4, "0")}_stats'   for seed in seeds])
plans_stack   = s.join([f'select * from {tbl}_{str(seed).rjust(4, "0")}_plans'   for seed in seeds])

query = f"""
select
    E.seed,
    E.plan,
    E.{district_type},
    max(E.pop_imbalance) as pop_imbalance,
    max(E.polsby_popper_plan) as polsby_popper_plan,
    max(E.polsby_popper_district) as polsby_popper_district,
    max(E.total_pop) as total_pop,
    sum(F.total_1race)
from (
    select
        B.seed,
        B.plan,
        C.{district_type},
        B.pop_imbalance,
        B.polsby_popper as polsby_popper_plan,
        C.polsby_popper as polsby_popper_district,
        C.total_pop,
        D.geoid
    from (
        select
            *
        from (
            select
                *,
                row_number() over (partition by A.hash order by plan asc, seed asc) as r
            from (
                {summary_stack}
                ) as A
            )
        where r = 1
        ) as B
    inner join (
        {stats_stack}
        ) as C
    on
        B.seed = C.seed and B.plan = C.plan
    inner join (
        select
            *
        from (
            {plans_stack}
            )
        ) as D
    on
        C.seed = D.seed and C.plan = D.plan and C.{district_type} = D.{district_type}
    ) as E
inner join
    {nodes} as F
on
    E.geoid = F.geoid
group by
    seed, plan, {district_type}
"""
df = run_query(query)
# 
df#['hash'].value_counts().sort_values(ascending=False)

,seed,plan,cd,pop_imbalance,polsby_popper_plan,polsby_popper_district,total_pop,f0_
0,111,9,02,55.718287,19.195283,12.457524,575591,488720
1,111,21,02,51.275447,17.592821,12.457524,575591,488720
2,111,22,04,51.275447,17.439941,14.673533,949693,849910
3,111,23,04,51.275447,17.282653,14.673533,949693,849910
4,111,28,07,50.307764,18.059005,27.321004,606907,517689
...,...,...,...,...,...,...,...,...
16563,113,1,22,88.179594,19.907970,14.200205,972269,833709
16564,113,8,22,88.179594,17.787377,14.200205,972269,833709
16565,108,1,22,88.179594,19.915859,14.200205,972269,833709
16566,106,7,22,84.423077,19.240416,14.200205,972269,833709


In [105]:
district_type = 'cd'

G.nodes_tbl = f'cmat-315920.redistricting_data.nodes_TX_2020_cntyvtd_cd'
M.tbl = f'cmat-315920.redistricting_results_cook.TX_2020_cntyvtd_cd_seed'

s = "\nunion all\n"
seeds = [s for s in range (100, 116) if s not in [104, 110]]
summary_stack = s.join([f'select * from {tbl}_{str(seed).rjust(4, "0")}_summary' for seed in seeds])
stats_stack   = s.join([f'select * from {tbl}_{str(seed).rjust(4, "0")}_stats'   for seed in seeds])
plans_stack   = s.join([f'select * from {tbl}_{str(seed).rjust(4, "0")}_plans'   for seed in seeds])

cols = [c for c in get_cols(self.nodes) if c not in Levels + District_types + ['geoid', 'county', 'total_pop', 'polygon', 'perim', 'polsby_popper', 'density', 'point']]

query = f"""
select
    B.seed,
    B.plan,
    C.{district_type},
    max(B.pop_imbalance) as pop_imbalance_plan,
    max(B.polsby_popper) as polsby_popper_plan,
    max(C.polsby_popper) as polsby_popper_district,
    max(C.total_pop) as total_pop,
    max(C.total_pop) / sum(E.aland) as density,
    {join_str(1).join([f'sum(E.{c}) as {c}' for c in cols])}
from (
    select
        *
    from (
        select
            *,
            row_number() over (partition by A.hash order by plan asc, seed asc) as r
        from (
            {summary_stack}
            ) as A
        )
    where r = 1
    ) as B
inner join (
    {stats_stack}
    ) as C
on
    B.seed = C.seed and B.plan = C.plan
inner join (
    select
        *
    from (
        {plans_stack}
        )
    ) as D
on
    C.seed = D.seed and C.plan = D.plan and C.{district_type} = D.{district_type}
inner join
    {nodes} as E
on
    D.geoid = E.geoid
group by
    seed, plan, {district_type}
"""
df = run_query(query)
# 
df#['hash'].value_counts().sort_values(ascending=False)

,seed,plan,cd,pop_imbalance_plan,polsby_popper_plan,polsby_popper_district,total_pop,density,density2,f0_
0,114,22,29,54.554121,18.883316,11.529660,789040,2541.736441,2541.736441,613241
1,108,5,01,79.840469,19.636886,19.227926,918940,82.639152,82.639152,839466
2,103,36,25,52.515885,19.705162,11.554691,783077,259.448137,259.448137,672360
3,105,22,10,49.759906,19.058262,22.359293,725883,146.075678,146.075678,608170
4,109,1,17,88.179594,19.672451,20.205281,644610,111.310852,111.310852,565132
...,...,...,...,...,...,...,...,...,...,...
16563,101,6,21,80.184022,20.549793,18.591359,847865,143.314010,143.314010,703978
16564,113,7,21,88.179594,17.957862,18.591359,847865,143.314010,143.314010,703978
16565,115,3,21,88.179594,20.576569,18.591359,847865,143.314010,143.314010,703978
16566,108,6,21,77.788805,19.670205,18.591359,847865,143.314010,143.314010,703978


In [116]:
file = '/home/jupyter/results/TX_2020_cntyvtd_cd_2021_09_05_01_50_26'
df = pd.read_csv(file+'.csv')

In [124]:
file = pathlib.Path('/home/jupyter/results/TX_2020_cntyvtd_cd_2021_09_05_01_50_26')
pq = file /'.parquet'
pq.mkdir(parents=True, exist_ok=True)
df.to_parquet(pq, index=False, partition_cols='seed')

In [79]:
query = f"""
    select
        *
    from (
        {stats_stack}
        )
"""
run_query(query)

,seed,plan,cd,polsby_popper,total_pop
0,114,0,01,19.400764,722363
1,114,1,01,19.400764,722363
2,114,2,01,19.400764,722363
3,114,3,01,19.400764,722363
4,114,4,01,19.400764,722363
...,...,...,...,...,...
17817,100,26,38,26.350145,882273
17818,100,27,38,26.350145,882273
17819,100,28,38,26.350145,882273
17820,100,29,38,26.350145,882273


In [67]:
df

,seed,plan,hash,pop_imbalance,polsy_popper,r
0,114,5,-7915478809046984484,67.880162,19.059284,1
1,104,5,-6709255699811009922,84.423077,20.401792,1
2,103,14,-6459736489883378198,65.249087,18.834906,1
3,108,7,-6393807558829545116,77.788805,19.582134,1
4,111,25,-5541735169537300084,50.307764,17.205912,1
...,...,...,...,...,...,...
450,108,35,-4037798534409500956,55.848145,18.065215,1
451,115,43,-2188271141291405578,52.394110,17.920852,1
452,107,17,756676115651280966,57.830578,18.232746,1
453,105,18,3543382720719122944,65.375295,18.984034,1


In [64]:
tbl = f'cmat-315920.redistricting_results_cook.TX_2020_cntyvtd_cd_seed'
sels = [f'select * from {tbl}_{str(seed).rjust(4, "0")}_summary' for seed in range (100, 116) if seed != 110]
query = "\nunion all\n".join(sels)
query = f"""
    select
        A.hash,
        max(seed),
        max(plan),
        count(*) as count
    from (
        {query}
        ) as A
    group by
        A.hash
"""
df = run_query(query)
# 
df['hash'].value_counts().sort_values(ascending=False)

-9000968703311639662    1
 1686731019043959074    1
 5326950720663711766    1
 3782232777236031018    1
-2692892709489472754    1
                       ..
-8883311163469327286    1
 4073746617601787200    1
 588406201666064194     1
 2579663012286536464    1
 2357127729327828404    1
Name: hash, Length: 455, dtype: int64

In [56]:
df.dtypes

seed               int64
plan               int64
hash               int64
pop_imbalance    float64
polsy_popper     float64
r                  int64
dtype: object

In [63]:
# df['seed'].unique()



-9000968703311639662    15
-3612179245007624834     1
 8819512477695847480     1
-8088312543324601526     1
-7104331499606189002     1
                        ..
 4209038618835388202     1
 7292070618630616890     1
-6120954419377883746     1
-5055405117699558834     1
 2731995706378505462     1
Name: hash, Length: 455, dtype: int64